In [2]:
# -----------------------------
# 1️⃣ Initialisation Spark
# -----------------------------
from pyspark.sql import SparkSession
import os

# Variables MinIO
os.environ["MINIO_ROOT_USER"] = "minio"
os.environ["MINIO_ROOT_PASSWORD"] = "minio123"
os.environ["MINIO_BUCKET"] = "telco-churn"

MINIO_ACCESS_KEY = os.environ["MINIO_ROOT_USER"]
MINIO_SECRET_KEY = os.environ["MINIO_ROOT_PASSWORD"]
MINIO_BUCKET = os.environ["MINIO_BUCKET"]

# SparkSession avec Delta et Hadoop AWS pour S3A
spark = SparkSession.builder \
    .appName("TelcoDataFlow_BatchToDelta") \
    .config("spark.hadoop.fs.s3a.access.key", MINIO_ACCESS_KEY) \
    .config("spark.hadoop.fs.s3a.secret.key", MINIO_SECRET_KEY) \
    .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.3.6,com.amazonaws:aws-java-sdk-bundle:1.12.301") \
    .getOrCreate()

print("✅ SparkSession démarrée avec succès !")

# -----------------------------
# 2️⃣ Liste des fichiers CSV dans MinIO
# -----------------------------
input_path = f"s3a://{MINIO_BUCKET}/raw/*.csv"

try:
    df_raw = spark.read.format("csv") \
        .option("header", "true") \
        .option("inferSchema", "true") \
        .load(input_path)
    
    print(f"Nombre de lignes chargées : {df_raw.count()}")
    df_raw.show(5)
except Exception as e:
    print("❌ Erreur lors de la lecture des fichiers CSV :", e)
    df_raw = None

# -----------------------------
# 3️⃣ Écriture dans Delta Lake
# -----------------------------
delta_path = "/datalake/telco_churn/"

if df_raw:
    try:
        df_raw.write.format("delta") \
            .mode("append") \
            .save(delta_path)
        print(f"✅ Données stockées dans Delta Lake : {delta_path}")
    except Exception as e:
        print("❌ Erreur lors de l'écriture dans Delta Lake :", e)

# -----------------------------
# 4️⃣ Aperçu / Statistiques
# -----------------------------
if df_raw:
    print("📊 Aperçu des données :")
    df_raw.show(5)
    df_raw.describe().show()


✅ SparkSession démarrée avec succès !
❌ Erreur lors de la lecture des fichiers CSV : An error occurred while calling o53.load.
: java.lang.NoClassDefFoundError: org/apache/hadoop/fs/impl/prefetch/PrefetchingStatistics
	at java.base/java.lang.ClassLoader.defineClass1(Native Method)
	at java.base/java.lang.ClassLoader.defineClass(ClassLoader.java:1017)
	at java.base/java.security.SecureClassLoader.defineClass(SecureClassLoader.java:150)
	at java.base/java.net.URLClassLoader.defineClass(URLClassLoader.java:524)
	at java.base/java.net.URLClassLoader$1.run(URLClassLoader.java:427)
	at java.base/java.net.URLClassLoader$1.run(URLClassLoader.java:421)
	at java.base/java.security.AccessController.doPrivileged(AccessController.java:712)
	at java.base/java.net.URLClassLoader.findClass(URLClassLoader.java:420)
	at java.base/java.lang.ClassLoader.loadClass(ClassLoader.java:592)
	at java.base/java.lang.ClassLoader.loadClass(ClassLoader.java:525)
	at org.apache.hadoop.fs.s3a.S3AFileSystem.initialize(

In [3]:
print(spark.version)
hadoop_version = spark._jsc.hadoopConfiguration().get("hadoop.version")
print(f"Hadoop version utilisée par Spark : {hadoop_version}")


3.5.0
Hadoop version utilisée par Spark : None


In [4]:
# -----------------------------
# 1️⃣ Initialisation Spark
# -----------------------------
from pyspark.sql import SparkSession
import os

# -----------------------------
# Variables MinIO
# -----------------------------
os.environ["MINIO_ROOT_USER"] = "minio"
os.environ["MINIO_ROOT_PASSWORD"] = "minio123"
os.environ["MINIO_BUCKET"] = "telco-churn"

MINIO_ACCESS_KEY = os.environ["MINIO_ROOT_USER"]
MINIO_SECRET_KEY = os.environ["MINIO_ROOT_PASSWORD"]
MINIO_BUCKET = os.environ["MINIO_BUCKET"]

# -----------------------------
# Chemins vers les jars Hadoop / AWS
# -----------------------------
hadoop_jars = [
    "/chemin/vers/jars/hadoop-common-3.3.6.jar",
    "/chemin/vers/jars/hadoop-aws-3.3.6.jar",
    "/chemin/vers/jars/aws-java-sdk-bundle-1.12.301.jar"
]
hadoop_jars_str = ",".join(hadoop_jars)

# -----------------------------
# SparkSession avec Delta et Hadoop AWS
# -----------------------------
spark = SparkSession.builder \
    .appName("TelcoDataFlow_BatchToDelta") \
    .config("spark.jars", hadoop_jars_str) \
    .config("spark.hadoop.fs.s3a.access.key", MINIO_ACCESS_KEY) \
    .config("spark.hadoop.fs.s3a.secret.key", MINIO_SECRET_KEY) \
    .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .getOrCreate()

print("✅ SparkSession démarrée avec succès !")

# -----------------------------
# 2️⃣ Lecture des CSV batch depuis MinIO
# -----------------------------
input_path = f"s3a://{MINIO_BUCKET}/raw/*.csv"

try:
    df_raw = spark.read.format("csv") \
        .option("header", "true") \
        .option("inferSchema", "true") \
        .load(input_path)
    
    print(f"Nombre de lignes chargées : {df_raw.count()}")
    df_raw.show(5)
except Exception as e:
    print("❌ Erreur lors de la lecture des fichiers CSV :", e)
    df_raw = None

# -----------------------------
# 3️⃣ Écriture dans Delta Lake
# -----------------------------
delta_path = "/datalake/telco_churn/"

if df_raw:
    try:
        df_raw.write.format("delta") \
            .mode("append") \
            .save(delta_path)
        print(f"✅ Données stockées dans Delta Lake : {delta_path}")
    except Exception as e:
        print("❌ Erreur lors de l'écriture dans Delta Lake :", e)

# -----------------------------
# 4️⃣ Analyse rapide
# -----------------------------
if df_raw:
    print("📊 Aperçu des données :")
    df_raw.show(5)
    df_raw.describe().show()


✅ SparkSession démarrée avec succès !
❌ Erreur lors de la lecture des fichiers CSV : An error occurred while calling o62.load.
: java.lang.NoClassDefFoundError: org/apache/hadoop/fs/impl/prefetch/PrefetchingStatistics
	at java.base/java.lang.ClassLoader.defineClass1(Native Method)
	at java.base/java.lang.ClassLoader.defineClass(ClassLoader.java:1017)
	at java.base/java.security.SecureClassLoader.defineClass(SecureClassLoader.java:150)
	at java.base/java.net.URLClassLoader.defineClass(URLClassLoader.java:524)
	at java.base/java.net.URLClassLoader$1.run(URLClassLoader.java:427)
	at java.base/java.net.URLClassLoader$1.run(URLClassLoader.java:421)
	at java.base/java.security.AccessController.doPrivileged(AccessController.java:712)
	at java.base/java.net.URLClassLoader.findClass(URLClassLoader.java:420)
	at java.base/java.lang.ClassLoader.loadClass(ClassLoader.java:592)
	at java.base/java.lang.ClassLoader.loadClass(ClassLoader.java:525)
	at org.apache.hadoop.fs.s3a.S3AFileSystem.initialize(